In [1]:
import numpy as np
import scipy.stats as stats
from utils import gibbs_sampling

In [2]:
# Model hyperparameters.
p = 2
K = 2

In [3]:
# Model parameters.
A1 = np.array([[0.29, 0.05], [0.32, 0.46]])
print('A1:')
print(A1)
A2 = np.array([[0.001, 0.15], [0.3, 0.34]])
print('A2:')
print(A2)
A = np.concatenate([A1, A2], axis=1)
print(A)
CovU = np.array([[9, 0], [0, 4]])
print('CovU')
print(CovU)

A1:
[[0.29 0.05]
 [0.32 0.46]]
A2:
[[0.001 0.15 ]
 [0.3   0.34 ]]
[[0.29  0.05  0.001 0.15 ]
 [0.32  0.46  0.3   0.34 ]]
CovU
[[9 0]
 [0 4]]


Chequeamos si cumple las condiciones requeridas:

In [4]:
#se chequea que la matriz CovU sea adecuada (semidefinida positiva)
eig_val_U = np.linalg.eigvals(CovU)
flag_sdp  = np.all(eig_val_U >= 0) and np.all(np.isreal(eig_val_U)) #valores propios no negativos y reales 
print(flag_sdp)

True


In [5]:
#se chequea que la matriz A sea adecuada (proceso estable, pag 15 Lutkepohl)
if(p==1): #no es necesario agregar bloque 
    A_test = A
else:
    A_block  = np.block(np.eye(K*(p-1)))
    A_zeros  = np.zeros((K*(p-1),K))
    A_bottom = np.concatenate((A_block,A_zeros),axis=1)
    A_test   = np.concatenate((A,A_bottom),axis=0)
eig_val_A   = np.absolute(np.linalg.eigvals(A_test))
print(eig_val_A)
flag_stable = np.all(eig_val_A < 1) #valores propios absolutos menores a 1
print(flag_stable)

[0.98621062 0.31920693 0.31920693 0.44443108]
True


In [6]:
# Real normalized data.
#[[0.87010704 0.89617185 0.94830146 1.10469029 1.1307551  1.18288471]
# [1.386914   1.54056019 1.61738329 1.64299099 1.66859869 1.66859869]]

In [7]:
# Initial values.
y0a = np.array([0.87010704, 1.386914])
y1a = np.array([0.89617185, 1.54056019])
y0b = np.array([0.94830146, 1.61738329])
y1b = np.array([1.10469029, 1.64299099])

In [8]:
def generate_data(A1, A2, covu, N, y0, y1):
    q = stats.multivariate_normal(mean=[0, 0], cov=covu)
    data = np.zeros((2, N))
    data[:, 0] = y0
    data[:, 1] = y1
    for i in range(2, N):
        new_y = np.dot(A1, y1) + np.dot(A2, y0) + q.rvs()
        data[:, i] = new_y
        y0 = y1
        y1 = new_y
    return data

In [9]:
# Fictional data.
data = generate_data(A1, A2, CovU, 1000, y0a, y1a)
#test_data_2 = generate_data(A1, A2, CovU, 1000, yb, yb)

In [10]:
# Prepare data.
X = np.zeros((K * p, data.shape[1] - p))
j = 0
for i in range(p, data.shape[1]):
    for t in range(p):
        X[t * K:(t + 1) * K, j] = data[:, (i - 1) - t]
    j += 1

Y0 = data[:, p:]

In [11]:
# Check data is right.
print(data.shape)
print(Y0.shape)
print(X.shape)

(2, 1000)
(2, 998)
(4, 998)


In [12]:
# Check data is right. x2
print(data[:, :6])
print(Y0[:, :6])
print(X[:, :6])

[[0.87010704 0.89617185 1.40250928 1.80786654 5.35489645 0.63605447]
 [1.386914   1.54056019 3.20291981 5.51482688 5.66968432 5.07624091]]
[[1.40250928 1.80786654 5.35489645 0.63605447 0.10125346 4.68282633]
 [3.20291981 5.51482688 5.66968432 5.07624091 5.15553306 5.39314347]]
[[0.89617185 1.40250928 1.80786654 5.35489645 0.63605447 0.10125346]
 [1.54056019 3.20291981 5.51482688 5.66968432 5.07624091 5.15553306]
 [0.87010704 0.89617185 1.40250928 1.80786654 5.35489645 0.63605447]
 [1.386914   1.54056019 3.20291981 5.51482688 5.66968432 5.07624091]]


In [13]:
# Run sampler.
DATA_PATH = '/home/chrisams/Documents/datasets/data_TAIM/processed/'
q = stats.norm
K = 2
p = 2
iters = 5
debug = False
mh_iters = 10
n_rows = 10000 # Number of rows of the data to load
method = 'normal'
init_mle = True
annealing = True
T0 = 500
TF = 1
annealing_n = 5
T = lambda t: T0 * ((TF / T0) ** (t / annealing_n))

samples = gibbs_sampling(iters, DATA_PATH, K, p, q, mh_iters=mh_iters, init_mle=init_mle, n_rows=None,\
                         debug=False, method='normal', X=X, Y0=Y0, annealing=annealing, T=T,\
                         annealing_n=annealing_n)

Initializing parameters...
lk init: -81299.27103141365
Calculating MLE...
Init MLE theta calculated! (-59732.01626181304)
[[-0.41602623  1.27914777 -0.11330126  0.07328838]
 [ 0.25252017 -0.2251658   0.46214763  0.54518681]]
[[7.98859903 0.8619511 ]
 [0.8619511  0.56787422]]
Iteration 0
accepted: 10.0%%
accepted: 20.0%%
accepted: 40.0%%
accepted: 20.0%%
accepted: 20.0%%
accepted: 10.0%%
accepted: 10.0%%
accepted: 0.0%%
accepted: 70.0%%
accepted: 60.0%%
accepted: 30.0%%
accepted: 40.0%%
LK of new theta: -6692.228375981051
Time for iteration 0: 0.5031588077545166
Iteration 1
accepted: 40.0%%
accepted: 0.0%%
accepted: 30.0%%
accepted: 20.0%%
accepted: 40.0%%
accepted: 10.0%%
accepted: 40.0%%
accepted: 10.0%%
accepted: 50.0%%
accepted: 40.0%%
accepted: 40.0%%
accepted: 30.0%%
LK of new theta: -3278.773749448134
Time for iteration 1: 0.49393749237060547
Iteration 2
accepted: 50.0%%
accepted: 40.0%%
accepted: 40.0%%
accepted: 30.0%%
accepted: 30.0%%
accepted: 10.0%%
accepted: 20.0%%
accepted

In [14]:
# Print for comparison.
for sample in samples:
    print(sample)
    print('-' * 20)

[array([[-0.72964384, -0.03052746,  0.13204231,  0.07328838],
       [ 0.33368739,  0.15377899,  0.44656863,  0.54518681]]), array([[6.22370714, 0.20887699],
       [0.2507407 , 0.71344667]])]
--------------------
[array([[ 0.55819008, -0.03052746, -0.02844332,  0.00955649],
       [ 0.33368739,  0.15377899,  0.44656863,  0.6202993 ]]), array([[ 4.34785706,  2.56315876],
       [-1.91816194,  2.09403251]])]
--------------------
[array([[ 1.07084015, -0.06233679, -0.52919588,  0.18450179],
       [ 0.19391048,  0.0635248 ,  0.44656863,  0.58757325]]), array([[ 3.81646078,  2.12044692],
       [-2.62355743,  2.30346454]])]
--------------------
[array([[ 0.73201654, -0.06233679, -0.52919588,  0.18450179],
       [ 0.47023028,  0.09971281,  0.44656863,  0.58757325]]), array([[ 4.69149835,  2.26892579],
       [-1.60556949,  2.52467395]])]
--------------------
[array([[ 0.94621514, -0.05189677, -0.55756058,  0.18450179],
       [ 0.27694427,  0.23202042,  0.44656863,  0.58757325]]), array([

In [15]:
# Model parameters.
A1 = np.array([[0.29, 0.05], [0.32, 0.46]])
print('A1:')
print(A1)
A2 = np.array([[0.001, 0.15], [0.3, 0.34]])
print('A2:')
print(A2)
A = np.concatenate([A1, A2], axis=1)
print(A)
CovU = np.array([[9, 0], [0, 4]])
print('CovU')
print(CovU)

A1:
[[0.29 0.05]
 [0.32 0.46]]
A2:
[[0.001 0.15 ]
 [0.3   0.34 ]]
[[0.29  0.05  0.001 0.15 ]
 [0.32  0.46  0.3   0.34 ]]
CovU
[[9 0]
 [0 4]]


In [16]:
np.random.choice([1, 3, 4])

1